In [1]:
from interface import ObjectInterface

FILENAME_DATABASE = 'data/database.db'

def get_interface() -> ObjectInterface:
    interface = ObjectInterface(FILENAME_DATABASE)
    interface.connect()
    return interface

In [5]:
source = """
from datetime import datetime
value = datetime.strptime(value, '%Y-%m-%d').date()
"""

with get_interface() as interface:
    obj = interface.get_object(1)
    print(obj.attributes)
    value = interface.execution_handler.transform_value(source, obj, obj.attributes['birthday'])
    print(value)

{'first_name': 'Anne', 'last_name': 'Fischer', 'birthday': '1997-03-25'}
1997-03-25


In [5]:
import utils
import numpy as np

a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = utils.array_to_bytes(a)
a_read = utils.bytes_to_array(b)
print(a, '\n', a_read)

[[1 2 3]
 [4 5 6]
 [7 8 9]] 
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
